# GBDT_ensemble

**reference:**

-[CatBoost Tutorial](https://github.com/catboost/catboost/blob/v0.25.1/catboost/tutorials/python_tutorial.ipynb)

## softmax feature

In [1]:
import pandas as pd
import pickle
import os

In [2]:
df1 = pd.read_csv(
    os.path.join("result", "bert-base-cased_bs_8_epo7", "l2", "level2_softmax.csv")
)
df2 = pd.read_csv(
    os.path.join("result", "roberta-base_bs_8_epo10", "l2", "level2_softmax.csv")
)
df3 = pd.read_csv(
    os.path.join("result", "xlnet-base-cased_bs_8_epo10", "l2", "level2_softmax.csv")
)
df4 = pd.read_csv(
    os.path.join(
        "result",
        "google_electra-base-discriminator_bs_8_epo10",
        "l2",
        "level2_softmax.csv",
    )
)
df5 = pd.read_csv(
    os.path.join(
        "result", "microsoft_deberta-base_bs_4_epo9", "l2", "level2_softmax.csv"
    )
)

In [3]:
df1["sentiment"]

0       0.025889
1       0.002131
2       0.004950
3       0.055823
4       0.002406
          ...   
2343    0.992563
2344    0.002670
2345    0.985187
2346    0.003030
2347    0.069894
Name: sentiment, Length: 2348, dtype: float64

In [4]:
with open(os.path.join("data", "train_valid_split", "level_2", "y_l2.pkl"), "rb") as f:
    y = pickle.load(f)

In [5]:
X = pd.DataFrame(
    {
        "model_1": df1["sentiment"],
        "model_2": df2["sentiment"],
        "model_3": df3["sentiment"],
        "model_4": df4["sentiment"],
        "model_5": df5["sentiment"],
    }
)
X

,model_1,model_2,model_3,model_4,model_5
0,0.025889,0.003011,0.004297,0.009604,0.005394
1,0.002131,0.002943,0.000889,0.002825,0.000381
2,0.004950,0.002949,0.000739,0.004916,0.001335
3,0.055823,0.002976,0.009089,0.003739,0.001982
4,0.002406,0.002948,0.000767,0.002968,0.000456
...,...,...,...,...,...
2343,0.992563,0.996553,0.997342,0.988435,0.998907
2344,0.002670,0.002945,0.000779,0.002885,0.000744
2345,0.985187,0.996455,0.995898,0.985997,0.997631
2346,0.003030,0.002957,0.000789,0.002995,0.004879


In [6]:
y

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

### Data Splitting

In [7]:
from sklearn.model_selection import train_test_split

X_tr, X_va, y_tr, y_va = train_test_split(X, y, train_size=0.8, random_state=0)

In [8]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

In [9]:
model = CatBoostClassifier(
    custom_loss=["Accuracy"],
    random_seed=0,
    logging_level="Silent",
)

In [11]:
model.fit(
    X_tr,
    y_tr,
    eval_set=(X_va, y_va),
    # logging_level='Verbose',  # you can uncomment this for text output
    plot=True,
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [12]:
cv_params = model.get_params()
cv_params.update({"loss_function": "Logloss"})
cv_data = cv(Pool(X, y), cv_params, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

## Hard-label feature

In [13]:
import pandas as pd
import pickle
import os

In [14]:
df1 = pd.read_csv(
    os.path.join("result", "bert-base-cased_bs_8_epo7", "l2", "level2.csv")
)
df2 = pd.read_csv(os.path.join("result", "roberta-base_bs_8_epo10", "l2", "level2.csv"))
df3 = pd.read_csv(
    os.path.join("result", "xlnet-base-cased_bs_8_epo10", "l2", "level2.csv")
)
df4 = pd.read_csv(
    os.path.join(
        "result",
        "google_electra-base-discriminator_bs_8_epo10",
        "l2",
        "level2.csv",
    )
)
df5 = pd.read_csv(
    os.path.join("result", "microsoft_deberta-base_bs_4_epo9", "l2", "level2.csv")
)

In [15]:
with open(os.path.join("data", "train_valid_split", "level_2", "y_l2.pkl"), "rb") as f:
    y = pickle.load(f)

In [16]:
X = pd.DataFrame(
    {
        "model_1": df1["sentiment"],
        "model_2": df2["sentiment"],
        "model_3": df3["sentiment"],
        "model_4": df4["sentiment"],
        "model_5": df5["sentiment"],
    }
)
X

,model_1,model_2,model_3,model_4,model_5
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
2343,1,1,1,1,1
2344,0,0,0,0,0
2345,1,1,1,1,1
2346,0,0,0,0,0


In [21]:
print(list(X.columns))

['model_1', 'model_2', 'model_3', 'model_4', 'model_5']


In [17]:
y.values

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

### Data Splitting

In [18]:
from sklearn.model_selection import train_test_split

X_tr, X_va, y_tr, y_va = train_test_split(X, y, train_size=0.8, random_state=0)

In [19]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

In [22]:
model = CatBoostClassifier(
    custom_loss=["Accuracy"],
    random_seed=0,
    logging_level="Silent",
)

In [23]:
model.fit(
    X_tr,
    y_tr,
    cat_features=list(X.columns),
    eval_set=(X_va, y_va),
    # logging_level='Verbose',  # you can uncomment this for text output
    plot=True,
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [24]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': 'Logloss'
})
cv_data = cv(
    Pool(X, y, cat_features=list(X.columns)),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))